In this build, we are building a xgboost model. 

The imputation methods are:
- Categorical: fill in with "NA"
- Numeric: fill in with interoplate linear

In [1]:
import pandas as pd
import scipy as sp
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
import xgboost as xgb
import pprint as pp
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import bnp_helper
import common_helper
import random
import os
import datetime
from sklearn.externals import joblib
from matplotlib.pylab import rcParams

%matplotlib inline
rcParams['figure.figsize'] = 16, 4
pd.set_option('display.max_columns', 500)

/Users/binlin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
################### Combine the training set and test set to fix the unmatched categories ############
train_df_raw = pd.read_csv("../data/train.csv")
test_df_raw = pd.read_csv("../data/test.csv")

df_raw_combined = pd.concat([train_df_raw, test_df_raw], axis = 0)

########### Clean and Impute (Combined) #############
df_combinded = bnp_helper.clean(df_raw_combined, drop_collinearity = True, inplace = True)
df_combinded = bnp_helper.impute_cate_with_na_numeric_with_outlier(df_combinded)

################ Convert text to number (Combined) ################
df_combinded = common_helper.dummify(df_combinded, bnp_helper.get_categorical_variables(df_combinded))
print df_combinded.shape
df_combinded.head()

(228714, 449)


,ID,target,v1,v10,v101,v102,v103,v106,v108,v109,v114,v117,v118,v119,v120,v122,v123,v124,v126,v127,v129,v13,v130,v131,v14,v16,v18,v19,v2,v21,v23,v27,v28,v35,v36,v37,v38,v4,v42,v43,v44,v45,v49,v50,v51,v53,v55,v57,v58,v59,v6,v60,v61,v62,v63,v65,v68,v69,v7,v70,v72,v73,v76,v77,v78,v80,v81,v82,v84,v85,v86,v87,v88,v9,v90,v93,v94,v95,v96,v98,v99,v107_B,v107_C,v107_D,v107_E,v107_F,v107_G,v107_NA,v110_B,v110_C,v112_B,v112_C,v112_D,v112_E,v112_F,v112_G,v112_H,v112_I,v112_J,v112_K,v112_L,v112_M,v112_N,v112_NA,v112_O,v112_P,v112_Q,v112_R,v112_S,v112_T,v112_U,v112_V,v113_AA,v113_AB,v113_AC,v113_AD,v113_AE,v113_AF,v113_AG,v113_AH,v113_AI,v113_AJ,v113_AK,v113_B,v113_C,v113_D,v113_E,v113_F,v113_G,v113_H,v113_I,v113_J,v113_K,v113_L,v113_M,v113_N,v113_NA,v113_O,v113_P,v113_Q,v113_R,v113_S,v113_T,v113_U,v113_V,v113_W,v113_X,v113_Y,v113_Z,v125_AA,v125_AB,v125_AC,v125_AD,v125_AE,v125_AF,v125_AG,v125_AH,v125_AI,v125_AJ,v125_AK,v125_AL,v125_AM,v125_AN,v125_AO,v125_AP,v125_AQ,v125_AR,v125_AS,v125_AT,v125_AU,v125_AV,v125_AW,v125_AX,v125_AY,v125_AZ,v125_B,v125_BA,v125_BB,v125_BC,v125_BD,v125_BE,v125_BF,v125_BG,v125_BH,v125_BI,v125_BJ,v125_BK,v125_BL,v125_BM,v125_BN,v125_BO,v125_BP,v125_BQ,v125_BR,v125_BS,v125_BT,v125_BU,v125_BV,v125_BW,v125_BX,v125_BY,v125_BZ,v125_C,v125_CA,v125_CB,v125_CC,v125_CD,v125_CE,v125_CF,v125_CG,v125_CH,v125_CI,v125_CJ,v125_CK,v125_CL,v125_D,v125_E,v125_F,v125_G,v125_H,v125_I,v125_J,v125_K,v125_L,v125_M,v125_N,v125_NA,v125_O,v125_P,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z,v24_B,v24_C,v24_D,v24_E,v3_B,v3_C,v3_NA,v30_B,v30_C,v30_D,v30_E,v30_F,v30_G,v30_NA,v31_B,v31_C,v31_NA,v47_B,v47_C,v47_D,v47_E,v47_F,v47_G,v47_H,v47_I,v47_J,v52_B,v52_C,v52_D,v52_E,v52_F,v52_G,v52_H,v52_I,v52_J,v52_K,v52_L,v52_NA,v56_AA,v56_AB,v56_AC,v56_AD,v56_AE,v56_AF,v56_AG,v56_AH,v56_AI,v56_AJ,v56_AK,v56_AL,v56_AM,v56_AN,v56_AO,v56_AP,v56_AQ,v56_AR,v56_AS,v56_AT,v56_AU,v56_AV,v56_AW,v56_AX,v56_AY,v56_AZ,v56_B,v56_BA,v56_BB,v56_BC,v56_BD,v56_BE,v56_BF,v56_BG,v56_BH,v56_BI,v56_BJ,v56_BK,v56_BL,v56_BM,v56_BN,v56_BO,v56_BP,v56_BQ,v56_BR,v56_BS,v56_BT,v56_BU,v56_BV,v56_BW,v56_BX,v56_BY,v56_BZ,v56_C,v56_CA,v56_CB,v56_CC,v56_CD,v56_CE,v56_CF,v56_CG,v56_CH,v56_CI,v56_CJ,v56_CK,v56_CL,v56_CM,v56_CN,v56_CO,v56_CP,v56_CQ,v56_CR,v56_CS,v56_CT,v56_CU,v56_CV,v56_CW,v56_CX,v56_CY,v56_CZ,v56_D,v56_DA,v56_DB,v56_DC,v56_DD,v56_DE,v56_DF,v56_DG,v56_DH,v56_DI,v56_DJ,v56_DK,v56_DL,v56_DM,v56_DN,v56_DO,v56_DP,v56_DQ,v56_DR,v56_DS,v56_DT,v56_DU,v56_DV,v56_DW,v56_DX,v56_DY,v56_DZ,v56_E,v56_F,v56_G,v56_H,v56_I,v56_J,v56_K,v56_L,v56_M,v56_N,v56_NA,v56_O,v56_P,v56_Q,v56_R,v56_S,v56_T,v56_U,v56_V,v56_W,v56_X,v56_Y,v56_Z,v66_B,v66_C,v71_B,v71_C,v71_D,v71_E,v71_F,v71_G,v71_H,v71_I,v71_J,v71_K,v71_L,v74_B,v74_C,v75_B,v75_C,v75_D,v79_B,v79_C,v79_D,v79_E,v79_F,v79_G,v79_H,v79_I,v79_J,v79_K,v79_L,v79_M,v79_N,v79_O,v79_P,v79_Q,v79_R,v91_B,v91_C,v91_D,v91_E,v91_F,v91_G,v91_NA
0,3,1,1.335739,0.503281,8.389237,2.757375,4.374296,12.579184,2.382692,3.930922,15.634907,6.592012,5.909091,-6.297423e-07,1.059603,8.000000,1.989780,0.035754,1.804126,3.113719,0,2.866830,0.636365,2.857144,11.636387,8.571429,0.106720,0.148883,8.727474,7.730923,-1.716131e-08,3.393503,0.590122,8.375452,11.326592,0.454546,0,3.921026,12.707581,2.015505,10.498338,9.848672,8.086643,0.899420,7.277792,16.747968,1.299638,3.971118,0.529802,10.890984,2.599278,1.588448,15.858152,1,0.153461,18.303925,15.231789,17.142857,3.176895,11.784549,1,1.614988,2.230940,7.292418,8.571429,3.000000,7.528326,8.861647,1.299638,1.707317,0.866426,9.551836,3.321300,9.999999,0.905342,5.814018,3.517720,0.462019,7.436824,8.877414,1.191337,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [28]:
################### Combine the training set and test set to fix the unmatched categories ############
train_df_raw = pd.read_csv("../data/train.csv")
test_df_raw = pd.read_csv("../data/test.csv")

df_raw_combined = pd.concat([train_df_raw, test_df_raw], axis = 0)

########### Clean and Impute (Combined) #############
df_combinded = bnp_helper.clean(df_raw_combined, drop_collinearity = True, inplace = True)
df_combinded = bnp_helper.impute_cate_with_na_numeric_with_outlier(df_combinded)

################ Convert text to number (Combined) ################
#df_combinded = common_helper.dummify(df_combinded, bnp_helper.get_categorical_variables(df_combinded))
print df_combinded.shape
df_combinded.head()

(228714, 99)


,ID,target,v107,v110,v112,v113,v125,v24,v3,v30,v31,v47,v52,v56,v66,v71,v74,v75,v79,v91,v1,v10,v101,v102,v103,v106,v108,v109,v114,v117,v118,v119,v120,v122,v123,v124,v126,v127,v129,v13,v130,v131,v14,v16,v18,v19,v2,v21,v23,v27,v28,v35,v36,v37,v38,v4,v42,v43,v44,v45,v49,v50,v51,v53,v55,v57,v58,v59,v6,v60,v61,v62,v63,v65,v68,v69,v7,v70,v72,v73,v76,v77,v78,v80,v81,v82,v84,v85,v86,v87,v88,v9,v90,v93,v94,v95,v96,v98,v99
0,3,1,E,B,O,NA,AU,C,C,C,A,C,G,DI,C,F,B,D,E,A,1.335739,0.503281,8.389237,2.757375,4.374296,12.579184,2.382692,3.930922,15.634907,6.592012,5.909091,-6.297423e-07,1.059603,8.000000,1.989780,0.035754,1.804126,3.113719,0,2.866830,0.636365,2.857144,11.636387,8.571429,0.106720,0.148883,8.727474,7.730923,-1.716131e-08,3.393503,0.590122,8.375452,11.326592,0.454546,0,3.921026,12.707581,2.015505,10.498338,9.848672,8.086643,0.899420,7.277792,16.747968,1.299638,3.971118,0.529802,10.890984,2.599278,1.588448,15.858152,1,0.153461,18.303925,15.231789,17.142857,3.176895,11.784549,1,1.614988,2.230940,7.292418,8.571429,3.000000,7.528326,8.861647,1.299638,1.707317,0.866426,9.551836,3.321300,9.999999,0.905342,5.814018,3.517720,0.462019,7.436824,8.877414,1.191337
1,4,1,B,A,U,G,AF,C,C,C,A,E,G,DY,A,F,B,D,D,B,-999.000000,1.312910,-999.000000,-999.000000,-999.000000,-999.000000,1.825361,4.247858,10.308044,10.595357,-999.000000,-9.990000e+02,-999.000000,-999.000000,-999.000000,0.598896,-999.000000,-999.000000,0,-999.000000,-999.000000,-999.000000,11.636386,-999.000000,-999.000000,-999.000000,-999.000000,6.763110,-9.990000e+02,-999.000000,-999.000000,-999.000000,14.579479,-999.000000,0,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,1.379210,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,2,2.544736,-999.000000,-999.000000,-999.000000,-999.000000,12.053353,2,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,7.277655,3.430691,-999.000000,-999.000000,-999.000000,9.848004,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,8.303967,-999.000000
2,5,1,C,B,S,NA,AE,E,C,NA,A,C,F,AS,A,B,B,B,E,G,0.943877,0.765864,5.879353,3.292788,5.924457,11.670572,1.375753,1.184211,11.205561,3.478911,6.233767,-2.792745e-07,2.138728,9.333333,2.477596,0.013452,1.773709,3.922193,2,2.505589,0.883118,1.176472,9.603542,5.882353,0.244541,0.144258,5.310079,5.245035,-2.785053e-07,5.306122,0.836005,7.959184,12.730517,0.259740,0,4.410969,12.346939,2.926830,8.897561,5.343819,6.836734,0.604504,9.637627,15.102041,0.765305,4.030613,4.277456,9.105481,3.979592,2.151361,16.075602,1,0.123643,16.377205,11.040463,5.882353,3.928571,8.460654,3,2.413618,1.963971,5.918368,11.764705,3.333334,10.194433,8.266200,1.530613,2.429906,1.071429,8.447465,3.367346,12.666667,0.811447,5.156559,4.214944,0.309657,5.663265,11.588858,0.841837
3,6,1,B,B,J,NA,CJ,D,C,C,B,C,H,BW,A,F,B,D,B,B,0.797415,6.542669,8.507281,2.503055,4.872157,12.554274,2.230754,1.990131,13.777666,4.949609,7.180722,5.655086e-01,1.166281,7.018256,1.812795,0.002267,1.415230,2.954381,1,3.903302,1.677108,1.034483,14.094723,5.517242,1.224114,0.231630,8.304757,7.517125,-4.805344e-07,2.303640,8.926662,8.898468,11.302795,0.433735,0,4.225930,12.935823,1.470878,12.708574,9.670823,8.591954,3.329176,4.780357,16.621695,1.178161,3.965517,1.732102,11.777912,2.097700,1.229246,15.927390,1,0.140260,17.011645,18.568129,9.425288,1.987549,13.594728,2,2.272541,2.188198,8.213602,13.448277,1.947261,4.797873,13.315819,1.379310,1.587045,1.242817,10.747144,1.408046,8.965516,1.042425,5.498902,3.423944,0.832518,7.375480,6.942002,1.334611
4,8,1,C,A,T,G,Z,E,C,NA,A,I,H,NA,C,F,B,D,C,G,-999.000000,1.050328,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,14.097099,-999.000000,-999.000000,-9.990000e+02,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0,-999.000000,-999.000000,-999.000000,10.991098,-999.000000,-999.000000,-999.000000,-999.000000,6.414567,-9.990000e+02,-999.000000,-999.000000,-999.000000,-999.

In [29]:
train_df = df_combinded[-df_raw_combined['target'].isnull()]
test_df = df_combinded[df_raw_combined['target'].isnull()]

train_df_sample = train_df.sample(1000, random_state = 0)
#train_df_sample = train_df
target_train = train_df_sample['target']
X_train = train_df_sample.drop(['ID', 'target'], axis = 1)

In [34]:
logit = LogisticRegression(random_state=27, n_jobs = -1)

logit.fit(X_train, target_train)

#predprob_train = logit.predict_proba(X_train)

#metrics.log_loss(target_train, predprob_train)

ValueError: could not convert string to float: B

In [11]:
train_df = df_combinded[-df_raw_combined['target'].isnull()]
test_df = df_combinded[df_raw_combined['target'].isnull()]

train_df_sample = train_df.sample(40000, random_state = 0)
#train_df_sample = train_df
target_train = train_df_sample['target']
X_train = train_df_sample.drop(['ID', 'target'], axis = 1)

In [18]:
logit = LogisticRegression(random_state=27, n_jobs = -1)

para_grid = [{'penalty': ['l1', 'l2'], 
              'fit_intercept': [False, True], 
              'C':np.logspace(-5, 5, 10)}]


start = datetime.datetime.now()
para_search = GridSearchCV(logit, para_grid, scoring='log_loss', cv =5).fit(X_train, target_train)
end = datetime.datetime.now()
print "model training time: {}".format(end - start)

pp.pprint(para_search.grid_scores_)

# best combinatin
print 'Best combination: ', para_search.best_params_

# best score
print 'Best Score' , para_search.best_score_

model training time: 1:19:51.735297
[mean: -0.62436, std: 0.00339, params: {'penalty': 'l1', 'C': 1.0000000000000001e-05, 'fit_intercept': False},
 mean: -0.54557, std: 0.00089, params: {'penalty': 'l2', 'C': 1.0000000000000001e-05, 'fit_intercept': False},
 mean: -0.62436, std: 0.00339, params: {'penalty': 'l1', 'C': 1.0000000000000001e-05, 'fit_intercept': True},
 mean: -0.54555, std: 0.00091, params: {'penalty': 'l2', 'C': 1.0000000000000001e-05, 'fit_intercept': True},
 mean: -0.54657, std: 0.00222, params: {'penalty': 'l1', 'C': 0.00012915496650148841, 'fit_intercept': False},
 mean: -0.53514, std: 0.00482, params: {'penalty': 'l2', 'C': 0.00012915496650148841, 'fit_intercept': False},
 mean: -0.54658, std: 0.00222, params: {'penalty': 'l1', 'C': 0.00012915496650148841, 'fit_intercept': True},
 mean: -0.53562, std: 0.00508, params: {'penalty': 'l2', 'C': 0.00012915496650148841, 'fit_intercept': True},
 mean: -0.52944, std: 0.00196, params: {'penalty': 'l1', 'C': 0.0016681005372000

In [21]:
#train_df_sample = train_df.sample(40000, random_state = 0)
train_df_sample = train_df
target_train = train_df_sample['target']
X_train = train_df_sample.drop(['ID', 'target'], axis = 1)

In [22]:
# best estimator
logit_best = para_search.best_estimator_
logit_best.fit(X_train, target_train)

<bound method LogisticRegression.score of LogisticRegression(C=0.27825594022071259, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l1', random_state=27,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)>

In [24]:
predprob_train = logit_best.predict_proba(X_train)
metrics.log_loss(target_train, predprob_train)

0.4933370539949325

In [27]:
joblib.dump(logit_best, 'save/logit_model.pkl')

['save/logit_model.pkl',
 'save/logit_model.pkl_01.npy',
 'save/logit_model.pkl_02.npy',
 'save/logit_model.pkl_03.npy',
 'save/logit_model.pkl_04.npy']